In [1]:
import pandas as pd
import re
import tarfile
import gzip
from bs4 import BeautifulSoup

In [2]:
!pwd

/home/jupyter/notebooks/archive.org/commercial


In [3]:
df = pd.read_json('commercial.json')
df

,archive_id,intervals,video_id
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5
...,...,...,...
304968,FOXNEWSW_20230715_060000_Hannity,"[[1151150, 1314296], [1870919, 1885900], [2450...",362930
304969,MSNBCW_20230714_100000_Morning_Joe,"[[1081980, 1340906], [2228526, 2480744], [3558...",362931
304970,CNNW_20230715_020000_Whos_Talking_to_Chris_Wal...,"[[0, 66866], [678077, 918317], [1439404, 16847...",362932
304971,FOXNEWSW_20230714_150000_The_Faulkner_Focus,"[[2746360, 3040520], [3345875, 3631010]]",362933


In [4]:
# tar tvzf html-2023.tar.gz > html-2017.txt
archive_id = []
with open('../shared/html-2023.txt') as f:
    for l in f:
        a = l.split('/')
        aid = a[-1].strip().replace('.html', '')
        archive_id.append(aid)

In [5]:
df2023 = pd.DataFrame({'archive_id': archive_id})
df2023

,archive_id
0,
1,GBN_20230601_140000_Patrick_Christys
2,KTVU_20230411_000000_KTVU_FOX_2_News_at_5pm
3,DW_20221031_000200_Euromaxx
4,NTV_20230321_130000_Segodnya
...,...
179802,IRINN_20230307_063000
179803,GBN_20230304_200000_Mark_Dolan_Tonight
179804,1TV_20230307_235500_PODKAST
179805,PRESSTV_20221110_010000_World_News_in_Full


In [6]:
cdf = df[df.archive_id.isin(df2023.archive_id)].copy()
cdf

,archive_id,intervals,video_id
287293,FOXNEWSW_20221027_000000_Tucker_Carlson_Tonight,"[[1119268, 1399097], [1739954, 2039954], [2548...",344885
287295,MSNBCW_20221027_000000_All_In_With_Chris_Hayes,"[[0, 49532], [1144209, 1288487], [1843174, 210...",344887
287329,CNNW_20221027_000000_Anderson_Cooper_360,"[[1010893, 1176959], [1806437, 1950164], [2335...",344921
287362,FOXNEWSW_20221027_130000_Americas_Newsroom,"[[768684, 1089288], [1571403, 1867282], [22842...",344955
287363,MSNBCW_20221027_020000_The_Last_Word_With_Lawr...,"[[1177459, 1451583], [1875506, 2154635], [2549...",344956
...,...,...,...
302286,FOXNEWSW_20230604_130000_FOX_and_Friends_Sunday,"[[0, 21254], [1251867, 1471636], [1761509, 201...",360177
302289,MSNBCW_20230604_140000_Velshi,"[[0, 52402], [1037669, 1123438], [1406972, 165...",360180
302291,MSNBCW_20230604_130000_The_Sunday_Show_With_Jo...,"[[930796, 1175340], [1600548, 1843491], [22010...",360182
302293,FOXNEWSW_20230604_140000_Sunday_Morning_Future...,"[[912328, 1132397], [1589304, 1829310], [24211...",360184


In [7]:
cdf['html_file'] = 'html-2023/' + cdf['archive_id'] + '.html'
cdf

,archive_id,intervals,video_id,html_file
287293,FOXNEWSW_20221027_000000_Tucker_Carlson_Tonight,"[[1119268, 1399097], [1739954, 2039954], [2548...",344885,html-2023/FOXNEWSW_20221027_000000_Tucker_Carl...
287295,MSNBCW_20221027_000000_All_In_With_Chris_Hayes,"[[0, 49532], [1144209, 1288487], [1843174, 210...",344887,html-2023/MSNBCW_20221027_000000_All_In_With_C...
287329,CNNW_20221027_000000_Anderson_Cooper_360,"[[1010893, 1176959], [1806437, 1950164], [2335...",344921,html-2023/CNNW_20221027_000000_Anderson_Cooper...
287362,FOXNEWSW_20221027_130000_Americas_Newsroom,"[[768684, 1089288], [1571403, 1867282], [22842...",344955,html-2023/FOXNEWSW_20221027_130000_Americas_Ne...
287363,MSNBCW_20221027_020000_The_Last_Word_With_Lawr...,"[[1177459, 1451583], [1875506, 2154635], [2549...",344956,html-2023/MSNBCW_20221027_020000_The_Last_Word...
...,...,...,...,...
302286,FOXNEWSW_20230604_130000_FOX_and_Friends_Sunday,"[[0, 21254], [1251867, 1471636], [1761509, 201...",360177,html-2023/FOXNEWSW_20230604_130000_FOX_and_Fri...
302289,MSNBCW_20230604_140000_Velshi,"[[0, 52402], [1037669, 1123438], [1406972, 165...",360180,html-2023/MSNBCW_20230604_140000_Velshi.html
302291,MSNBCW_20230604_130000_The_Sunday_Show_With_Jo...,"[[930796, 1175340], [1600548, 1843491], [22010...",360182,html-2023/MSNBCW_20230604_130000_The_Sunday_Sh...
302293,FOXNEWSW_20230604_140000_Sunday_Morning_Future...,"[[912328, 1132397], [1589304, 1829310], [24211...",360184,html-2023/FOXNEWSW_20230604_140000_Sunday_Morn...


In [8]:
html_files = cdf['html_file'].tolist()
html_files[:20]

['html-2023/FOXNEWSW_20221027_000000_Tucker_Carlson_Tonight.html',
 'html-2023/MSNBCW_20221027_000000_All_In_With_Chris_Hayes.html',
 'html-2023/CNNW_20221027_000000_Anderson_Cooper_360.html',
 'html-2023/FOXNEWSW_20221027_130000_Americas_Newsroom.html',
 'html-2023/MSNBCW_20221027_020000_The_Last_Word_With_Lawrence_ODonnell.html',
 'html-2023/CNNW_20221027_080000_CNN_Newsroom_Live.html',
 'html-2023/FOXNEWSW_20221027_090000_FOX__Friends_First.html',
 'html-2023/FOXNEWSW_20221027_100000_FOX_and_Friends.html',
 'html-2023/FOXNEWSW_20221027_070000_The_Ingraham_Angle.html',
 'html-2023/FOXNEWSW_20221027_150000_The_Faulkner_Focus.html',
 'html-2023/CNNW_20221027_030000_CNN_Tonight.html',
 'html-2023/CNNW_20221027_150000_At_This_Hour_With_Kate_Bolduan.html',
 'html-2023/CNNW_20221027_110000_New_Day_With_John_Berman_and_Brianna_Keilar.html',
 'html-2023/FOXNEWSW_20221027_180000_America_Reports.html',
 'html-2023/MSNBCW_20221027_030000_The_11th_Hour_With_Stephanie_Ruhle.html',
 'html-2023/FOX

In [9]:
def is_in_intervals(intervals, start, end):
    for (a, b) in intervals:
        if (a <= start <= b) or (a <= end <= b):
            return True
    return False

In [10]:
%%time

i = 0
data = []
ofn = 'archive-cc-2023-nc.csv.gz'
first = True
with tarfile.open('../shared/html-2023.tar.gz', 'r:gz') as tar:
    for tar_info in tar:
        if tar_info.isfile():
            if tar_info.name in html_files:
                fname = tar_info.name
                i += 1
                if i % 100 == 0:
                    # output to file every 100 records
                    print(i, fname)
                    tdf = pd.DataFrame(data)
                    data = []
                    if first:
                        tdf.to_csv(ofn, index=False, header=first, compression='gzip')
                        first = False
                    else:
                        tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')
                try:
                    fobj = tar.extractfile(tar_info)
                    html = fobj.read()
                    soup = BeautifulSoup(html, 'html.parser')
                    html = soup.prettify()
                    soup = BeautifulSoup(html, 'html.parser')
                    intervals = cdf[cdf.html_file==tar_info.name]['intervals'].iloc[0]
                    text = ""
                    for a in soup.find_all('div', {'class': 'tvcol'}):
                        l = a.find('a')
                        href = l.attrs['href']
                        m = re.match('/start/(\d+)/end/(\d+)', href)
                        if m:
                            start = int(m.group(1)) * 1000
                            end = int(m.group(2)) * 1000
                            t = a.find('div', {'class': 'snipin nosel'})
                            if not is_in_intervals(intervals, start, end):
                                text += t.text.strip()
                    archive_id = fname.replace('html-2023/', '').replace('.html', '')        
                    data.append({'archive_id': archive_id, 'text_nc': text})
                except Exception as e:
                    print(e, fname)

# last chunk
tdf = pd.DataFrame(data)
tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')

100 html-2023/CNNW_20230506_010000_Anderson_Cooper_360.html
200 html-2023/MSNBCW_20230212_080000_Dateline.html
300 html-2023/FOXNEWSW_20221116_030000_The_Ingraham_Angle.html
400 html-2023/MSNBCW_20230316_080000_Alex_Wagner_Tonight.html
500 html-2023/MSNBCW_20230415_190000_Yasmin_Vossoughian_Reports.html
600 html-2023/FOXNEWSW_20230425_160000_Outnumbered.html
700 html-2023/CNNW_20230328_100000_CNN_This_Morning.html
800 html-2023/MSNBCW_20221114_030000_The_Turning_Point_From_Devils_Breath.html
900 html-2023/FOXNEWSW_20230425_010000_Hannity.html
1000 html-2023/CNNW_20230325_160000_CNN_Newsroom_With_Fredricka_Whitfield.html
1100 html-2023/FOXNEWSW_20230425_110000_FOX_and_Friends.html
1200 html-2023/FOXNEWSW_20230307_070000_Hannity.html
1300 html-2023/MSNBCW_20230504_200000_Deadline_White_House.html
1400 html-2023/MSNBCW_20230107_140000_Velshi.html
1500 html-2023/MSNBCW_20230114_170000_Alex_Witt_Reports.html
1600 html-2023/CNNW_20221219_130000_CNN_This_Morning.html
1700 html-2023/CNNW_20230